In [1]:
"""Loading transactions from an external system

Demonstration of loading a transaction XML file from another "External System" into LUSID.

Attributes
----------
cocoon
transactions
holdings
"""

'Loading transactions from an external system\n\nDemonstration of loading a transaction XML file from another "External System" into LUSID.\n\nAttributes\n----------\ncocoon\ntransactions\nholdings\n'

# Loading an "External System" Transaction XML into LUSID

The purpose of this notebook  is to demonstrate how a user could load a transaction XML file from another "External System" into LUSID. The notebook will walk through the following steps: <br>

(1) Load transaction XML into DataFrame. <br>
(2) Scan for portfolio code in DataFrame. If portfolio code from DataFrame does not exist in LUSID, create the portfolio in LUSID. <br>
(3) Translate the transaction data in DataFrame into data types which LUSID will recognise (e.g. transform "strings" to "datetimes"). <br>
(4) Use client identifiers to create placeholder LUIDS (these can be enriched at a later date once we receive the "External System" instrument master file). <br>
(5) Load the transaction into LUSID using the Transactions API. <br><br>


## Setup: Load libraries and create our LUSID API client


In [2]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
import globalfund as global_fund_tools
import lusidtools.cocoon.cocoon as cocoon_tools
from lusidjam import RefreshingToken
import fbnsdkutilities.utilities as utils

# Import Libraries
import pprint
from datetime import datetime, timedelta, time, date
import pytz
import uuid
import printer as prettyprint
from datetime import datetime
import pandas as pd
import numpy as np
import os
import json
import xml.etree.ElementTree as ET
import io

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = utils.ApiClientFactory(
    lusid,
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print('LUSID Client Initialised')
print('LUSID version : ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Client Initialised
LUSID version :  0.6.11244.0


As part of the setup, we need to define a Scope. Scopes in LUSID are used to partition data. In this example, we will use the Scope "EXT_SYSTEM" to represent an external trading system. 

In [3]:
scope = "EXT_SYSTEM"

## Step 1: Scrape Transaction XML data into a Python DataFrame

First we need to load the XML file into a Python DataFrame. This will enable us to work with the data, then pass it to the relevant APIs.

In [4]:
# Load the XML file into memory

ext_system_file = os.path.join("data", "ext_system_file.xml")

In [5]:
# Create a Python object from the XML file
# List out the individual tags which will need to be mapped

tree = ET.parse(ext_system_file)
root = tree.getroot()
list_tags = list(set([elem.tag for elem in root.iter()]))
for tag in list_tags:
    print(tag)

TRD_PRICE
TRD_TOUCH_COUNT
PORT_NAME
TRD_TRADE_DATE
TRD_NUMBER
INST_GROUP
TRD_COUNTERPARTY
TRD_TRANS_TYPE
INST_DESC
TRD_TRADER
TRANSACTIONS
TRD_LOCATION
CLIENT_ID
TRD_QUANTITY
TRADE
TRD_SETTLE_DATE
INST_TYPE
TRD_CURRENCY
INST_TICKER
TRD_PRINCIPAL


Create a list of the tags which we want to load into LUSID. For the purposes of this example, we will select a subset of fields from the sample XML file. In production, you may need to load more.

In [6]:
list_tags = [
    
    "TRD_TRANS_TYPE",
    "TRD_QUANTITY",
    "TRD_TRADE_DATE",
    "PORT_NAME",
    "INST_GROUP",
    "TRD_CURRENCY",
    "TRD_TOUCH_COUNT",
    "INST_DESC",
    "INST_TYPE",
    "TRD_PRINCIPAL",
    "TRD_TRADER",
    "TRD_PRICE",
    "TRD_COUNTERPARTY",
    "INST_TICKER",
    "CLIENT_ID",
    "TRD_SETTLE_DATE",
    "TRD_LOCATION",
    "TRD_NUMBER"
    
]

Iterate over the "External System" transaction XML and scrape the results into a Pandas DataFrame. We scrape the results into a DataFrame for two reasons:<br>

(1) We can use the DataFrame methods to transform the data into a format LUSID likes. <br>
(2) We can then use the SDKs to pass the DataFrame directly to LUSID<br>
<br>

In [7]:
# Define an empty list which will be used to create a DataFrame from XML results 
rows= []

# Iterate over the XML to find transactions in the sub-XML nested under the "TRADE" tag.
# Store each individual transaction as a dictionary.
# Then add each dictionary (representing a transaction) to a list.
# Use this list of dictionaries to create a DataFrame.

for trade in root.iter('TRADE'):
    
    trade_list = []
    for tag in list_tags:
        
        try:       
            trade_list.append((trade.find(tag).text))
        except AttributeError:
            trade_list.append(None)

    rows.append({"TRD_TRANS_TYPE": trade_list[0], 
                 "TRD_QUANTITY": trade_list[1], 
                 "TRD_TRADE_DATE": trade_list[2],
                 "PORT_NAME": trade_list[3],
                 "INST_GROUP": trade_list[4],
                 "TRD_CURRENCY": trade_list[5],
                 "TRD_TOUCH_COUNT": trade_list[6],
                 "INST_DESC": trade_list[7],
                 "INST_TYPE": trade_list[8],
                 "TRD_PRINCIPAL": trade_list[9],
                 "TRD_TRADER": trade_list[10],
                 "TRD_PRICE": trade_list[11],
                 "TRD_COUNTERPARTY": trade_list[12],
                 "INST_TICKER": trade_list[13],
                 "CLIENT_ID": trade_list[14],
                 "TRD_SETTLE_DATE": trade_list[15],
                 "TRD_LOCATION": trade_list[16],
                 "TRD_NUMBER": trade_list[17]
                
                })
    
ext_system_df = pd.DataFrame(rows)

Let's take a look at the DataFrame. The results look good. We see trade details - broker, security type, price etc. 

In [8]:
ext_system_df

,TRD_TRANS_TYPE,TRD_QUANTITY,TRD_TRADE_DATE,PORT_NAME,INST_GROUP,TRD_CURRENCY,TRD_TOUCH_COUNT,INST_DESC,INST_TYPE,TRD_PRINCIPAL,TRD_TRADER,TRD_PRICE,TRD_COUNTERPARTY,INST_TICKER,CLIENT_ID,TRD_SETTLE_DATE,TRD_LOCATION,TRD_NUMBER
0,BUY,55000.0000000000,8/16/2018,TEST_PORT1,EQUITY,USD,2,INTERNATIONAL BUSINESS MACHINES CORP,COMMON_STOCK,7864450.000,TEST_TRADER,142.99,GS,IBM,EXT1YU6H5,8/20/2018,LONDON,0011223
1,BUY,100000.0000000000,9/16/2018,TEST_PORT1,EQUITY,USD,1,BANK OF AMERICA CORP,COMMON_STOCK,2899000.000,TEST_TRADER,28.99,JPM,BAC,EXT1YU6H6,9/20/2018,LONDON,0011224
2,SELL,100000.0000000000,10/16/2018,TEST_PORT2,EQUITY,USD,1,WALMART INC,COMMON_STOCK,11699000,TEST_TRADER,116.99,JPM,WMT,EXT1YU6H7,10/20/2018,LONDON,0011225
3,BUY,100000.0000000000,5/16/2018,TEST_PORT2,EQUITY,USD,6,3M CO,COMMON_STOCK,11599000,TEST_TRADER,155.99,MSUSA,MMM,EXT1YU6H8,5/20/2018,LONDON,0011226


## Step 2: Create portfolios in LUSID

Next we need to create portfolios in LUSID. The "External System" transaction XMLs will be loaded into these portfolio.

In [9]:
# Create a list of the unique portfolios from the transaction XML file

portfolios = list(set(ext_system_df["PORT_NAME"].tolist()))

In [10]:
# Prepare a DataFrame of portfolios

rows = []
for port in portfolios:
    
    rows.append(
        
        {'FundCode': port,
         'display_name': 'External System Positions {}'.format(port),
         'created': '2016-10-09',
         'base_currency': 'USD',
        'description': 'A portfolio to hold trades from an external system - {}'.format(port)}
        
    )
    

portfolios_data = pd.DataFrame(rows)
portfolios_data

,FundCode,display_name,created,base_currency,description
0,TEST_PORT2,External System Positions TEST_PORT2,2016-10-09,USD,A portfolio to hold trades from an external sy...
1,TEST_PORT1,External System Positions TEST_PORT1,2016-10-09,USD,A portfolio to hold trades from an external sy...


In [11]:
portfolio_mapping_required = {
  'code': 'FundCode',
  'display_name': 'display_name',
  'base_currency': 'base_currency'
}

portfolio_mapping_optional = {
  'created': 'created',
  'description': 'description',
  'accounting_method': None
}

In [12]:
# Create the portfolios if they do not already exist in LUSID

responses = cocoon_tools.load_from_data_frame(
    api_factory=api_factory, 
    scope=scope, 
    data_frame=portfolios_data, 
    mapping_required=portfolio_mapping_required, 
    mapping_optional=portfolio_mapping_optional,
    file_type='portfolio')

for response in responses["portfolios"]["success"]:
    prettyprint.portfolio_response(response)

Portfolio Created
Scope: EXT_SYSTEM
Code: TEST_PORT2
Portfolio Effective From: 2016-10-09 00:00:00+00:00
Portfolio Created On: 2023-05-04 10:38:40.668204+00:00

Portfolio Created
Scope: EXT_SYSTEM
Code: TEST_PORT1
Portfolio Effective From: 2016-10-09 00:00:00+00:00
Portfolio Created On: 2023-05-04 10:38:40.668204+00:00



## Step 3: Translate the data into a LUSID-friendly format

The script above loads "Trade Date" and "Settle Date" as strings. LUSID needs these as datetimes.

In [13]:
ext_system_df['TRD_TRADE_DATE'] = pd.to_datetime(
    ext_system_df['TRD_TRADE_DATE'], format="%m/%d/%Y").apply(lambda x: pytz.utc.localize(x))

ext_system_df['TRD_SETTLE_DATE'] = pd.to_datetime(
    ext_system_df['TRD_SETTLE_DATE'], format="%m/%d/%Y").apply(lambda x: pytz.utc.localize(x))

In [14]:
ext_system_df

,TRD_TRANS_TYPE,TRD_QUANTITY,TRD_TRADE_DATE,PORT_NAME,INST_GROUP,TRD_CURRENCY,TRD_TOUCH_COUNT,INST_DESC,INST_TYPE,TRD_PRINCIPAL,TRD_TRADER,TRD_PRICE,TRD_COUNTERPARTY,INST_TICKER,CLIENT_ID,TRD_SETTLE_DATE,TRD_LOCATION,TRD_NUMBER
0,BUY,55000.0000000000,2018-08-16 00:00:00+00:00,TEST_PORT1,EQUITY,USD,2,INTERNATIONAL BUSINESS MACHINES CORP,COMMON_STOCK,7864450.000,TEST_TRADER,142.99,GS,IBM,EXT1YU6H5,2018-08-20 00:00:00+00:00,LONDON,0011223
1,BUY,100000.0000000000,2018-09-16 00:00:00+00:00,TEST_PORT1,EQUITY,USD,1,BANK OF AMERICA CORP,COMMON_STOCK,2899000.000,TEST_TRADER,28.99,JPM,BAC,EXT1YU6H6,2018-09-20 00:00:00+00:00,LONDON,0011224
2,SELL,100000.0000000000,2018-10-16 00:00:00+00:00,TEST_PORT2,EQUITY,USD,1,WALMART INC,COMMON_STOCK,11699000,TEST_TRADER,116.99,JPM,WMT,EXT1YU6H7,2018-10-20 00:00:00+00:00,LONDON,0011225
3,BUY,100000.0000000000,2018-05-16 00:00:00+00:00,TEST_PORT2,EQUITY,USD,6,3M CO,COMMON_STOCK,11599000,TEST_TRADER,155.99,MSUSA,MMM,EXT1YU6H8,2018-05-20 00:00:00+00:00,LONDON,0011226


## Step 4: Create placeholder LUIDs from "External System" client IDs

In an ideal world, we would setup our instrument master *before* upserting transactions. Unfortunately, that is not always possible. In this example, the external system or party has sent us the transaction file before an instrument master file. Therefore, as a workaround, we will map the unique CLIENT_IDs to a placeholder LUSID Unique Identifer (called a LUID). These LUIDs can be updated with more data (Public IDs, Sectors, Ratings etc) when the client "Instrument Master" file is sent to us later in the day.<br>

In [15]:
# Create a DataFrame containing the "External System" ID

instruments = ext_system_df[["CLIENT_ID", "INST_DESC"]]
instruments

,CLIENT_ID,INST_DESC
0,EXT1YU6H5,INTERNATIONAL BUSINESS MACHINES CORP
1,EXT1YU6H6,BANK OF AMERICA CORP
2,EXT1YU6H7,WALMART INC
3,EXT1YU6H8,3M CO


In [16]:
instrument_mapping_required = {
  'name': 'INST_DESC'
}

instrument_identifier_mapping = {
    'ClientInternal': 'CLIENT_ID'
}

instrument_mapping_optional = {}

In [17]:
response = cocoon_tools.load_from_data_frame(
    api_factory=api_factory, 
    data_frame=instruments, 
    identifier_mapping=instrument_identifier_mapping, 
    mapping_required=instrument_mapping_required,
    mapping_optional=instrument_mapping_optional,
    file_type='instruments',
    scope='ExtSysProperties001')

prettyprint.instrument_response(response["instruments"]["success"][0])

Instruments Successfully Upserted: 


,Instrument,ClientInternal ID,LUSID Instrument ID
0,ClientInternal: EXT1YU6H5,EXT1YU6H5,LUID_SDFNWK4K
1,ClientInternal: EXT1YU6H8,EXT1YU6H8,LUID_JCULLL40
2,ClientInternal: EXT1YU6H7,EXT1YU6H7,LUID_WEKTCJ2N
3,ClientInternal: EXT1YU6H6,EXT1YU6H6,LUID_8NAZNNXZ


## Step 5: Load transactions into LUSID

Finally, we want to load the transactions into LUSID. 

In [18]:
transaction_field_mapping_required = {
    "code": "PORT_NAME",
    "transaction_id": "TRD_NUMBER",
    "type": 'TRD_TRANS_TYPE',
    "transaction_date": 'TRD_TRADE_DATE',
    "settlement_date": 'TRD_SETTLE_DATE',
    "units": "TRD_QUANTITY",
    "transaction_price.price": "TRD_PRICE",
    "transaction_price.type": "$Price",
    "total_consideration.amount": "TRD_PRINCIPAL",
    "total_consideration.currency": "TRD_CURRENCY",
    "transaction_currency": "TRD_CURRENCY"
    }

transaction_field_mapping_optional = {
    "exchange_rate": "$1",
    "source": f"${scope}"
}

transaction_identifier_mapping = {
    'ClientInternal': "CLIENT_ID"
}

In [19]:
responses = cocoon_tools.load_from_data_frame(
    api_factory=api_factory, 
    scope=scope, 
    data_frame=ext_system_df,
    mapping_required=transaction_field_mapping_required, 
    mapping_optional=transaction_field_mapping_optional,
    identifier_mapping=transaction_identifier_mapping,
    file_type="transaction")

for response in responses["transactions"]["success"]:
    print ('\n')
    prettyprint.transactions_response(response, scope, response.href.split("/")[7])



Transactions Successfully Upserted into Portfolio
Scope: EXT_SYSTEM
Code: TEST_PORT1
Transactions Effective From: 2016-10-09 00:00:00+00:00
Transactions Created On: 2019-10-11 08:23:28.335641+00:00



Transactions Successfully Upserted into Portfolio
Scope: EXT_SYSTEM
Code: TEST_PORT2
Transactions Effective From: 2016-10-09 00:00:00+00:00
Transactions Created On: 2019-10-11 08:23:28.718814+00:00



The transactions are now visible in the front-end LUSID tool.

![Init](img/read_transactions.png)